## Pytorch Study Image
This notebook is intended as a study of the basics of PyTorch, specificaly when dealing with images.  
I will first build a Neural Network with fully connected layers to classify the MNIST dataset. Then I will use a Convolutional Neural Network for classification of the CIFAR-10 dataset.  
Note that this is not intended to have a great performance, but to get a better understanding of how PyTorch works, so I will use only some basic tools and will not perform any optimization on the parameters, rather I will use the predefined settings or use some commonly used parameters.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn #API for building neural networks
from torch.utils.data import Dataset, DataLoader #Imports the Dataset and Dataloader classes
import os
import matplotlib.pyplot as plt

### Dataset

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device)) #Use GPU if available

In [ ]:
#Required hyperparameters
learning_rate = 0.001
epochs = 30

Finally, we can call these functions to train and evaluate our model.

Note how the test score starts higher than the train score and the test loss lower than the train loss. That is beacause of the batch training.  
For training, we calculate the average over all the batches, but after every batch we update the weights, so, in principle (and as demonstrated by the graph), the model learns something after each batch. So after each epoch we have already updated the model 120 times (60000 samples/500 batch) to calculate the test score and loss.

### Image data
Although we used the MNIST dataset above, we used only linear layers. For images it is usual to use convolutional layers, so the model can capture spatial patterns.  
Also, the MNIST dataset consists of grayscale images, but in most cases we will deal with color images, so below we will construct a convolutional neural network for classification of color images using the CIFAR-10 dataset.

First we will create the dataset. We need to create a .csv file with the names of the files and its respective label.

With this code I have created the annotations for the training and test images

    import csv
    
    train_folder = '../input/cifar10-pngs-in-folders/cifar10/train'  
    test_folder = '../input/cifar10-pngs-in-folders/cifar10/test'  
    labels_dict = {  
    'airplane': 0,  
    'horse': 1,  
    'truck': 2,  
    'automobile': 3,  
    'ship': 4,  
    'dog': 5,  
    'bird': 6,  
    'frog': 7,  
    'cat': 8,  
    'deer': 9  
    }  

    with open('../train_annotations.csv', mode='w') as csv_file:  
        csv_writer = csv.writer(csv_file)  
        for folder in os.listdir(train_folder):  
            for file in os.listdir(train_folder + '/' + folder):  
                label = str(labels_dict[folder])  
                path = train_folder + '/' + folder + '/' + file  
                csv_writer.writerow([path, label])  

    with open('../test_annotations.csv', mode='w') as csv_file:  
        csv_writer = csv.writer(csv_file)  
        for folder in os.listdir(train_folder):  
            for file in os.listdir(train_folder + '/' + folder):  
                label = str(labels_dict[folder])  
                path = train_folder + '/' + folder + '/' + file  
                csv_writer.writerow([path, label])

In [ ]:
from torchvision.io import read_image

class ImageDataset(Dataset):
    def __init__(self, annotations_file):
        self.annotations = pd.read_csv(annotations_file, header=None, 
                                       names=['Path', 'Label'], delimiter=',')
    
    def __len__(self):
        return(len(self.annotations))
    
    def __getitem__(self, index):
        path = self.annotations['Path'][index]
        label = torch.tensor(self.annotations['Label'][index]).float()
        img = read_image(path).float()
        return(img, label)

In [ ]:
train_dataset = ImageDataset('../input/cifarannotations/train_annotations.csv')
test_dataset = ImageDataset('../input/cifarannotations/test_annotations.csv')

We want to zero mean our dataset, so we will computer the mean for each channel 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device)) #Use GPU if available

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=500, shuffle=True, num_workers=8)

In [ ]:
ch0, ch1, ch2 = 0,0,0
for img, label in train_loader:
    ch0 += img[:,0,:,:].mean(dim=0)
    ch1 += img[:,1,:,:].mean(dim=0)
    ch2 += img[:,2,:,:].mean(dim=0)
div = len(train_dataset)/train_loader.batch_size
ch0 = ch0/div
ch1 = ch1/div
ch2 = ch2/div
mean_img = torch.stack((ch0,ch1,ch2)).to(device)

In [ ]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.ConvNet = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, padding=2), #32 x 32 x 20
        nn.ReLU(),
        nn.MaxPool2d(2, 2), # 16 x 16 x 20
        nn.Conv2d(in_channels=10, out_channels=10, kernel_size=5, padding=2), #16 x 16 x 20 
        nn.ReLU(),
        nn.Conv2d(in_channels=10, out_channels=10, kernel_size=5, padding=2), #16 x 16 x 20 
        nn.ReLU(),
        nn.MaxPool2d(2, 2), # 8 x 8 x 20
        nn.Flatten(),
        nn.Linear(8*8*10, 10) #First number is number of inputs, second is the number of outputs
        )
    def forward(self, x):
        logits = self.ConvNet(x)
        return(logits)

In [ ]:
model = NeuralNetwork().to(device) #Needs to be stored somewhere. Use GPU for speed.
print(model)

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return(total_params)
    
total_parameters = count_parameters(model)

In [ ]:
model_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def train_model(model, dataloader, model_loss, optimizer, mean=0):
    size = len(dataloader.dataset)
    total_loss = 0
    total_correct = 0
    model.train()
    for X, y in dataloader: #Gets batch each iteration until it runs out of data
        X = X.to(device)
        X = X - mean
        y = y.to(device)
        #Forward pass
        pred = model(X) #Returns the logits of the last layer
        loss = model_loss(pred, y.long())
        total_loss += loss
        total_correct += (pred.argmax(1) == y).sum().item()
        #Backward pass
        optimizer.zero_grad() #Sets all the gradients to 0, so it can be computed for this epoch
        loss.backward() #Backpropagates through the loss function
        optimizer.step() #Backpropagates and updates the weights of the model
        
    avg_loss = total_loss/size
    score = total_correct/size
    return(avg_loss, score)
        
def test_model(model, dataloader, model_loss, mean=0):
    size = len(dataloader.dataset)
    total_loss = 0
    total_correct = 0
    model.eval()
    for X,y in dataloader:
        X = X.to(device)
        X = X-mean
        y = y.to(device)
        pred = model(X) #Logits of the last layer
        loss = model_loss(pred, y.long())
        total_loss += loss.item() #This .item() get the value in the tensor. Avoids memory consuptiom
        total_correct += (pred.argmax(1) == y).sum().item() #by not storing the computational graph 
    avg_loss = total_loss/size
    score = total_correct/size
    return(avg_loss, score)

def train_and_test(model, train_loader, test_loader, model_loss, optimizer, mean, epochs):
    train_losses, train_scores, test_losses, test_scores = [], [], [], []
    for epoch in range(epochs):
        print('Epoch:', epoch)
        train_loss, train_score = train_model(model, train_loader, model_loss, optimizer, mean_img)
        test_loss, test_score = test_model(model, test_loader, model_loss, mean_img)
        train_losses.append(train_loss)
        train_scores.append(train_score)
        test_losses.append(test_loss)
        test_scores.append(test_score)
        print('Train loss: {:.2f}       Test loss: {:.2f}'.format(train_loss, test_loss))
        print('Train score: {:.2f}%    Test score: {:.2f}%'.format(train_score*100, test_score*100))
        print('='*20)

    return(train_losses, test_losses, train_scores, test_scores)

In [ ]:
#Plot the train and test errors and losses
def plot_trainxtest(train_losses, test_losses, train_scores, test_scores):
    fig, axs = plt.subplots(2,1, sharex=True)
    axs[0].plot(train_scores, label='Train')
    axs[0].plot(test_scores, label='Test')
    axs[0].legend(loc='lower right')
    axs[0].set_ylabel('Score')
    axs[1].plot(train_losses)
    axs[1].plot(test_losses)
    axs[1].set_xlabel('Epoch')
    axs[1].set_ylabel('Loss')
    fig.suptitle('Train x Test')
    fig.subplots_adjust(hspace = .001)
    axs[0].set_xticklabels(())
    axs[0].title.set_visible(False)
    fig.show()
    print('Final test score: ', test_scores[-1])

In [ ]:
epochs = 20
plot_trainxtest(*train_and_test(model, train_loader, test_loader, model_loss, optimizer, mean_img, epochs))

In [ ]:
torch.save(model.state_dict(), '.\CIFAR_model')